In [2]:
%pip install pandas requests beautifulsoup4 lxml selenium webdriver_manager

Note: you may need to restart the kernel to use updated packages.


## Installing necessary packages and creating needed functions

In [3]:
import pandas as pd
import numpy as np

import time

# BeatifulSoup packages
import requests 
from bs4 import BeautifulSoup 
import io
from io import StringIO 
import re

# Selenium packages

import selenium 
from selenium import webdriver 

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service 

from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By 


import pickle 
def save_object(obj, filename): 
    with open(filename, 'wb') as output: 
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# Collecting base links with BeatifulSoup

In [4]:
url = "https://www.gov.pl/web/gov/katalog-jednostek"
webpage_html = requests.get(url) 

soup = BeautifulSoup(webpage_html.text, 'html.parser')

hrefs = []
for a in soup.find_all('a', class_='accordion-link'): 
    text = a.get_text(strip=True)
    if text.startswith("Polska w") | text.startswith("Polska na"):
        hrefs.append(a['href'])


hrefs = [link for link in hrefs]
print(hrefs)

['https://www.gov.pl/albania', 'https://www.gov.pl/algieria', 'https://www.gov.pl/angola', 'https://www.gov.pl/arabiasaudyjska', 'https://www.gov.pl/argentyna', 'https://www.gov.pl/armenia', 'https://www.gov.pl/australia', 'https://www.gov.pl/austria', 'https://www.gov.pl/azerbejdzan', 'https://www.gov.pl/belgia', 'https://www.gov.pl/bialorus', 'https://www.gov.pl/bosniaihercegowina', 'https://www.gov.pl/brazylia', 'https://www.gov.pl/bulgaria', 'https://www.gov.pl/chile', 'https://www.gov.pl/chorwacja', 'https://www.gov.pl/chiny', 'https://www.gov.pl/cypr', 'https://www.gov.pl/czarnogora', 'https://www.gov.pl/czechy', 'https://www.gov.pl/dania', 'https://www.gov.pl/egipt', 'https://www.gov.pl/estonia', 'https://www.gov.pl/etiopia', 'https://www.gov.pl/filipiny', 'https://www.gov.pl/finlandia', 'https://www.gov.pl/francja', 'https://www.gov.pl/grecja', 'https://www.gov.pl/gruzja', 'https://www.gov.pl/hiszpania', 'https://www.gov.pl/holandia', 'https://www.gov.pl/indie', 'https://www.go

# Contact part
### part conducted by Aliaksandra Labko

In [5]:
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("start-maximized")  
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
driver_chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


kontakt_xpath = ("//div[contains(text(),'Kontakt')]"
                 " | "
                 "//div[contains(text(),'Ambasada')]")


ambasada_xpath = (
    "//body[@id='body']/main/div[contains(@class,'article-area main-container')]"
    "/article[@id='main-content']/div[@class='editor-content']/div/div/p[1]"
    " | "
    "//body[@id='body']/main/div[contains(@class,'article-area main-container')]"
    "/article[@id='main-content']/div[@class='editor-content']/div[1]"
    " | "
    "//body[@id='body']/main/div[contains(@class,'article-area main-container')]"   #this one is for Bulgaria but it still doesn't work:(
    "/article[@id='main-content']/div[@class='editor-content']/div[2]/p[1]"
)

texts = []
current_urls = []

for link in hrefs[0:50]:
    driver_chrome.get(link)  # Open each link
    time.sleep(4)  # Wait for page to load

    try:
        # Wait for the "Kontakt" button and click it
        WebDriverWait(driver_chrome, 10).until(EC.element_to_be_clickable((By.XPATH, kontakt_xpath)))
        driver_chrome.find_element(By.XPATH, kontakt_xpath).click()
        
        current_url = driver_chrome.current_url #i'm getting currently open link to use it in df_ATE later for country names
        current_urls.append(current_url)
        
       # print(f"Clicked 'Kontakt' on {link}")
        
        ambasada = driver_chrome.find_element("xpath", ambasada_xpath)
        texts.append(ambasada.text)
        
        print(ambasada)

        time.sleep(np.random.chisquare(1) + 4)  # Random delay

    except Exception as e:
        print(f"Could not find 'Kontakt' button on {link} or 'ambasada_xpath': {e}")
        
driver_chrome.quit()

# Countries [0:50] - 7 min 36 seconds

<selenium.webdriver.remote.webelement.WebElement (session="a59e42f05f48c733bbc67a2014a1d934", element="f.CEC3E3413776AC5F3FECCEAA32004127.d.4C6D7752BBBD1AA66D1672B8B247E37C.e.61")>
Could not find 'Kontakt' button on https://www.gov.pl/algieria or 'ambasada_xpath': Message: element click intercepted: Element <div class="title">...</div> is not clickable at point (472, 570). Other element would receive the click: <div class="main-container">...</div>
  (Session info: chrome=135.0.7049.115)
Stacktrace:
0   chromedriver                        0x0000000105256a54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x000000010524ecf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000104d9a864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000104de6f3c cxxbridge1$string$len + 406076
4   chromedriver                        0x0000000104de54a8 cxxbridge1$string$len + 399272
5   chromedriver                        0x00000001

In [46]:
# print(texts)

['adres: Rruga e Bogdanëve 50; 1001 Tirana\ntelefon: +355 445 100 20\ne-mail: tirana.amb.sekretariat@msz.gov.pl', 'adres: Rue Olof Palme, Nouveau-Paradou, Hydra - Algier\ntelefon: +213 21 60 99 50\nfaks: +213 21 60 99 59\ne-mail: algier.amb.sekretariat@msz.gov.pl', 'adres: Rua Damiao de Gois 64, C.P. 1340, Luanda, Republika Angoli\ntelefon: +244 222 32 71 99\ne-mail: luanda.amb.sekretariat@msz.gov.pl', 'adres: Riyadh 12253, Al-Wurud Area, Hamran Street, Building No. 2307 PO Box 94016\ntelefon: +966 11 4708519, +966 11 4549089, +966 11 4508889\nfaks: +966 11 4549210\ne-mail: rijad.amb.sekretariat@msz.gov.pl', 'adres: Alejandro María de Aguado 2870, C1425CEB Buenos Aires, Argentyna', ' ', 'adres: Azerbejdżan, AZ-1000, Baku ul. Kichik Gala 2, Icheri Sheher (Stare Miasto)\ntelefon: +994 12 505 53 60\nfaks: +994 12 492 02 14\ne-mail: baku.amb.sekretariat@msz.gov.pl', 'adres: Avenue des Gaulois 29, 1040 Etterbeek\ntelefon: +32 2 7804500 (nie dotyczy informacji w sprawach konsularnych)\ne-mai

In [6]:

address_pattern = r"adres:\s*(.+)"
phone_pattern = re.compile(
    r"(?:telefon:|sekretariat:|telefony Ambasady:)\s*([\+\d\s]+)"
)
email_pattern = r"e-mail:\s*([\w\.-]+@[\w\.-]+)"


# Prepare a list to hold the extracted data
data = []

# Iterate through each entry in the texts list
for text in texts:
    # Apply regex to extract data
    address_match = re.search(address_pattern, text)
    phone_match = re.search(phone_pattern, text)
    email_match = re.search(email_pattern, text)

    # Store the extracted values (default to "N/A" if not found)
    address = address_match.group(1) if address_match else "N/A"
    phone = phone_match.group(1).strip() if phone_match else "N/A"
    email = email_match.group(1) if email_match else "N/A"

    # Append the result as a list of extracted values
    data.append([address, phone, email])

# Create a DataFrame
df_ATE = pd.DataFrame(data, columns=["Polish embassy address", "Phone Number", "E-mail"])

# Show the DataFrame
print(df_ATE)

                               Polish embassy address        Phone Number  \
0                   Rruga e Bogdanëve 50; 1001 Tirana     +355 445 100 20   
1   Rua Damiao de Gois 64, C.P. 1340, Luanda, Repu...    +244 934 112 210   
2   Alejandro María de Aguado 2870, C1425CEB Bueno...   + 54 11 4808 1700   
3            Hanrapetutyan 44/1, 0010 Erywań, Armenia    +374 10 54 24 91   
4             Hietzinger Hauptstraße 42C, 1130 Wiedeń     +43 1 87015 834   
5   Azerbejdżan, AZ-1000, Baku ul. Kichik Gala 2, ...   +994 12 505 53 60   
6                                                 N/A                 N/A   
7    Višnjik 20, 71000 Sarajewo, Bośnia i Hercegowina     +387 33 290 501   
8   Av. das Nações Qd. 809, Lote 33, CEP: 70423-90...    +55 61 3212 8000   
9                                                 N/A                 N/A   
10  Mar del Plata 2055, Providencia, Santiago de C...     +56 22 824 5000   
11                        Miramarska 23, 10000 Zagreb    +385 1 48 99 444   

## Creating Data Frame for Contact info

In [7]:
countries = [a.split('/')[4].capitalize() for a in current_urls] #getting coutries names from open link upon clicking on 'kontakt'
df_ATE.insert(loc=0, column= 'Country', value=countries) #i'm limiting it to 14 bcs in previous step 'kontakt' button wasn't found for Aurtalia, this limitaiton can be removed once we scrap whole site
print(df_ATE)

               Country                             Polish embassy address  \
0              Albania                  Rruga e Bogdanëve 50; 1001 Tirana   
1               Angola  Rua Damiao de Gois 64, C.P. 1340, Luanda, Repu...   
2            Argentyna  Alejandro María de Aguado 2870, C1425CEB Bueno...   
3              Armenia           Hanrapetutyan 44/1, 0010 Erywań, Armenia   
4              Austria            Hietzinger Hauptstraße 42C, 1130 Wiedeń   
5          Azerbejdzan  Azerbejdżan, AZ-1000, Baku ul. Kichik Gala 2, ...   
6             Bialorus                                                N/A   
7   Bosniaihercegowina   Višnjik 20, 71000 Sarajewo, Bośnia i Hercegowina   
8             Brazylia  Av. das Nações Qd. 809, Lote 33, CEP: 70423-90...   
9             Bulgaria                                                N/A   
10               Chile  Mar del Plata 2055, Providencia, Santiago de C...   
11           Chorwacja                        Miramarska 23, 10000 Zagreb   

# Travel information part
### conducted by Cezary Kuźmowicz

### Creating XPaths for all needed information

In [8]:
# Unfortunately, many subpages differ from each other - some starts with capital letter, some has element "li" in code while other "p".
# That's why I needed to adjust my code for such exceptions 

# 1. Passport info

passport_xpath = """
(//h3[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'wjazd i pobyt')]
  /following::li[
    contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'paszport')]
  |
  //h3[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'wjazd i pobyt')]
  /following::p[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'paszport')]
)
"""

# 2. Temporary passport info

temporary_passport_xpath = """
(//h3[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'wjazd i pobyt')]
  /following::li[
    (contains(translate(., 
          'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
          'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
        'paszport tymczasowy')
      or
      contains(translate(., 
          'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
          'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
        'paszport tymaczsowy'))]
  |
  //h3[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'wjazd i pobyt')]
  /following::p[(contains(translate(., 
          'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
          'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
        'paszport tymczasowy')
      or
      contains(translate(., 
          'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
          'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
        'paszport tymaczsowy'))]
)
"""

# 3. ID card info (dowód osobisty)


id_card_xpath = """
(//h3[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'wjazd i pobyt')]
  /following::li[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'dowód osobisty')]
  |
  //h3[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'wjazd i pobyt')]
  /following::p[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'dowód osobisty')]
)
"""

### "Special" countries

In [9]:
# some coutries decided to be very special and make my work harder by having pretty different HTML code
# there are only 2 of them, so I will do it rather manually 

# Our naughty boys are Norway and Peru

special_passport_xpath = """
(//strong[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'na jakim dokumencie podróży można przekroczyć granicę')]
  /following::li[contains(translate(., 
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'paszport')]
)
"""

special_temporary_passport_xpath = """
(//strong[contains(translate(.,
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'na jakim dokumencie podróży można przekroczyć granicę')]
  /following::li[(contains(translate(.,
          'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
          'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
        'paszport tymczasowy'))]
)
"""


special_id_card_xpath = """
(//strong[contains(translate(.,
        'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
        'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
      'na jakim dokumencie podróży można przekroczyć granicę')]
  /following::li[(contains(translate(.,
          'ABCDEFGHIJKLMNOPQRSTUVWXYZĘÓĄŚŁŻŹĆŃ',
          'abcdefghijklmnopqrstuvwxyzęóąśłżźćń'),
        'dowód osobisty'))]
)
"""



### Defining needed functions and other important things

In [10]:
# not to write that looooong function every time...
def random_sleep():
    time.sleep(np.random.chisquare(1) + 2)


# I found out that instead of clicking button "Informacje dla podrónych" each time, we can skip that step by modifying already collected links
# by attaching "/informacje-dla-podrozujacych" in the end we get our goal output

new_hrefs = []
for link in hrefs:  
    clean_link = link.rstrip("/")
    if "chiny" in clean_link.lower(): #we make one expection for China because their subpage is built different than others
        info_link = "https://www.gov.pl/web/chiny/idp"
    else:
        info_link = clean_link + "/informacje-dla-podrozujacych"
    new_hrefs.append(info_link)

## Scraping travel information

In [11]:
# setting up the scraper

chromepath = ChromeDriverManager().install()

service_chrome = Service(executable_path=chromepath)
options_chrome = webdriver.ChromeOptions()
driver_chrome = webdriver.Chrome(service=service_chrome, options=options_chrome)


# let fun begin! :D

result_data = []  

for link in new_hrefs[0:50]:
    driver_chrome.get(link)
    driver_chrome.maximize_window()
    random_sleep()

    # in some cases, after entering website we see list: main country + smaller ones
    # we decided to only scrap information about main countries to match them with contact info
    # (scrapping info about ALL ALL countries is idea for next project :P)

    try:
        sub_countries_xpath = "//article//div[@class='title']"
        sub_countries_list = driver_chrome.find_elements(By.XPATH, sub_countries_xpath)

        if len(sub_countries_list) > 1:
            print(f"[INFO] Found {len(sub_countries_list)} countries. Clicking only the main one.")
            sub_countries_list[0].click()
            random_sleep()
    except Exception as e:
        print(f"[WARN] Issue when checking additional country list on page: {link}")
        print(e)

    # for our 'special' countries we use different XPaths. For the rest - standard ones
    try:
        if "norwegia" in link.lower():
            passport_text = driver_chrome.find_element(By.XPATH, special_passport_xpath).text
            temporary_passport_text = driver_chrome.find_element(By.XPATH, special_temporary_passport_xpath).text
            id_card_text = driver_chrome.find_element(By.XPATH, special_id_card_xpath).text
            random_sleep()
        
        elif "peru" in link.lower():
            passport_text = driver_chrome.find_element(By.XPATH, special_passport_xpath).text
            temporary_passport_text = driver_chrome.find_element(By.XPATH, special_temporary_passport_xpath).text
            id_card_text = driver_chrome.find_element(By.XPATH, special_id_card_xpath).text
            random_sleep()
        
        else:
            passport_text = driver_chrome.find_element(By.XPATH, passport_xpath).text
            temporary_passport_text = driver_chrome.find_element(By.XPATH, temporary_passport_xpath).text
            id_card_text = driver_chrome.find_element(By.XPATH, id_card_xpath).text
            random_sleep()

        # saving information into dictionary
        data_dict = {
            "url": link,
            "passport": passport_text,
            "temporary_passport": temporary_passport_text,
            "id_card": id_card_text,
        }
        result_data.append(data_dict)
        
        print(f"[OK] Data collected for {link}")
        print(data_dict)
        
    except Exception as e:
        print(f"[ERROR] Could not retrieve 'Informacje dla podróżujących' elements on page {link}")
        print(e)
        continue

driver_chrome.quit()

# creating simple DataFrame
travel_info = pd.DataFrame(result_data)
travel_info

# 51 countries - 6 min 14 secons

[OK] Data collected for https://www.gov.pl/albania/informacje-dla-podrozujacych
{'url': 'https://www.gov.pl/albania/informacje-dla-podrozujacych', 'passport': 'Paszport TAK', 'temporary_passport': 'Paszport tymczasowy TAK', 'id_card': 'Dowód osobisty TAK'}
[INFO] Found 3 countries. Clicking only the main one.
[OK] Data collected for https://www.gov.pl/algieria/informacje-dla-podrozujacych
{'url': 'https://www.gov.pl/algieria/informacje-dla-podrozujacych', 'passport': 'Paszport TAK', 'temporary_passport': 'Paszport tymczasowy TAK', 'id_card': 'Dowód osobisty NIE'}
[INFO] Found 6 countries. Clicking only the main one.
[OK] Data collected for https://www.gov.pl/angola/informacje-dla-podrozujacych
{'url': 'https://www.gov.pl/angola/informacje-dla-podrozujacych', 'passport': 'paszport - TAK', 'temporary_passport': 'paszport tymaczsowy - NIE', 'id_card': 'dowód osobisty - NIE'}
[INFO] Found 3 countries. Clicking only the main one.
[OK] Data collected for https://www.gov.pl/arabiasaudyjska/in

,url,passport,temporary_passport,id_card
0,https://www.gov.pl/albania/informacje-dla-podr...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty TAK
1,https://www.gov.pl/algieria/informacje-dla-pod...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty NIE
2,https://www.gov.pl/angola/informacje-dla-podro...,paszport - TAK,paszport tymaczsowy - NIE,dowód osobisty - NIE
3,https://www.gov.pl/arabiasaudyjska/informacje-...,Paszport TAK,"Paszport tymczasowy TAK, przy wyjeździe",Dowód osobisty NIE
4,https://www.gov.pl/argentyna/informacje-dla-po...,Paszport: TAK,Paszport tymczasowy: TAK*,Dowód osobisty: NIE
5,https://www.gov.pl/armenia/informacje-dla-podr...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty NIE
6,https://www.gov.pl/australia/informacje-dla-po...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty NIE
7,https://www.gov.pl/austria/informacje-dla-podr...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty TAK
8,https://www.gov.pl/azerbejdzan/informacje-dla-...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty NIE
9,https://www.gov.pl/belgia/informacje-dla-podro...,Paszport TAK,Paszport tymczasowy TAK,Dowód osobisty TAK


## Cleaning Contact info DataFrame

In [12]:
# Firstly, renaming columns

raw_50 = travel_info
raw_50 = raw_50.rename(columns = {'url':'Country', 'passport':'Passport', 'temporary_passport':'Temporary Passport', 'id_card':'ID card'})

# "re" package wasn't working for me so I had to use simple function as below :(
# Sasha used it properly in her code 

# Secondly, deleting name of each document from 3 columns
raw_50["Passport"] = (
    raw_50["Passport"]
    .str.replace(r"(?i)\bpaszport\b", "", regex=True)  
    .str.replace(r"[:\-\–\*\ ]", "", regex=True)
    .str.strip()
)

raw_50["Temporary Passport"] = (
    raw_50["Temporary Passport"]
    .str.replace(r"(?i)\bpaszport\s+(?:tymczasowy|tymaczsowy)\b", "", regex=True)
    .str.replace(r"[:\-\–\*]", "", regex=True)
    .str.strip()
)

raw_50["ID card"] = (
    raw_50["ID card"]
    .str.replace(r"(?i)\bdowód\s+osobisty\b", "", regex=True)
    .str.replace(r"[:\-\–\*]", "", regex=True)
    .str.strip()
)


better_50 = raw_50

# Thirdly, extracting the name of the Country from the link
better_50["Country"] = (
    better_50["Country"]
    .str.extract(r"gov\.pl/([^/]+)/informacje-dla-podrozujacych", expand=False)
    .str.title() # changing from "albania" to "Albania"
)
# some countries like Arabia or Bosnia are not presented properly. We know it. 
# But managing it would be only manual work. Idea for the future ;)


# as being said before - China had different link. That's why we have to insret it manually
better_50.loc[16,'Country'] = 'Chiny'
ready_50 = better_50
ready_50

,Country,Passport,Temporary Passport,ID card
0,Albania,TAK,TAK,TAK
1,Algieria,TAK,TAK,NIE
2,Angola,TAK,NIE,NIE
3,Arabiasaudyjska,TAK,"TAK, przy wyjeździe",NIE
4,Argentyna,TAK,TAK,NIE
5,Armenia,TAK,TAK,NIE
6,Australia,TAK,TAK,NIE
7,Austria,TAK,TAK,TAK
8,Azerbejdzan,TAK,TAK,NIE
9,Belgia,TAK,TAK,TAK


## Merging 2 DataFrames

In [13]:
all_info = pd.merge(ready_50, df_ATE, on="Country", how="left")
all_info
# final DataFrame - voila!

,Country,Passport,Temporary Passport,ID card,Polish embassy address,Phone Number,E-mail
0,Albania,TAK,TAK,TAK,Rruga e Bogdanëve 50; 1001 Tirana,+355 445 100 20,tirana.amb.sekretariat@msz.gov.pl
1,Algieria,TAK,TAK,NIE,NaN,NaN,NaN
2,Angola,TAK,NIE,NIE,"Rua Damiao de Gois 64, C.P. 1340, Luanda, Repu...",+244 934 112 210,luanda.amb.sekretariat@msz.gov.pl
3,Arabiasaudyjska,TAK,"TAK, przy wyjeździe",NIE,NaN,NaN,NaN
4,Argentyna,TAK,TAK,NIE,"Alejandro María de Aguado 2870, C1425CEB Bueno...",+ 54 11 4808 1700,buenosaires.amb.sekretariat@msz.gov.pl
5,Armenia,TAK,TAK,NIE,"Hanrapetutyan 44/1, 0010 Erywań, Armenia",+374 10 54 24 91,erywan.amb.sekretariat@msz.gov.pl
6,Australia,TAK,TAK,NIE,NaN,NaN,NaN
7,Austria,TAK,TAK,TAK,"Hietzinger Hauptstraße 42C, 1130 Wiedeń",+43 1 87015 834,wieden.amb.sekretariat@msz.gov.pl
8,Azerbejdzan,TAK,TAK,NIE,"Azerbejdżan, AZ-1000, Baku ul. Kichik Gala 2, ...",+994 12 505 53 60,baku.amb.sekretariat@msz.gov.pl
9,Belgia,TAK,TAK,TAK,NaN,NaN,NaN


In [ ]:
# saving output to CSV

all_info.to_csv('travel_and_contact_info.csv', index = False)